<h1 style="color:#65AE11;">Multiple GPUs</h1>

CUDA can manage multiple GPU devices on a single host. In this section you will learn how.

<h2 style="color:#65AE11;">Objectives</h2>

By the time you complete this section you will understand the CUDA syntax to:

* Get how many GPUs are available to your application
* Activate any of the available GPUs
* Allocate memory on multiple GPUs
* Transfer memory to and from multiple GPUs
* Launch kernels on multiple GPUs

<h2 style="color:#65AE11;">Getting Information About Multiple GPUs</h2>

To computationally obtain the number of available GPUs available use `cudaGetDeviceCount`:

```c
int num_gpus;
cudaGetDeviceCount(&num_gpus);
```

To programmatically obtain the currently active GPU use `cudaGetDevice`:

```c
int device;
cudaGetDevice(&device); // `device` is now a 0-based index of the current GPU.
```

<h2 style="color:#65AE11;">Setting the Current GPU</h2>

For each host thread, one GPU device is active at a time. To set a specific GPU as active use `cudaSetDevice` with the desired GPU's 0-based index:

```c
cudaSetDevice(0);
```

<h2 style="color:#65AE11;">Looping Over Available GPUs</h2>

A common pattern is to loop over available GPUs, performing operations for each:

```c
int num_gpus;
cudaGetDeviceCount(&num_gpus);

for (int gpu = 0; gpu < num_gpus; gpu++) {

    cudaSetDevice(gpu);
    
    // Perform operations for this GPU.
}    
```

<h2 style="color:#65AE11;">Data Chunking for Multiple GPUs</h2>

(*If you need to review how to do robust indexing for data chunks, please see [Copy Compute Considerations](../08_Copy_Compute_Considerations/Copy_Compute_Considerations.ipynb).*)

As with multiple non-default streams, each of multiple GPUs can work with a chunk of data. Here we create and utilize an array of data pointers to allocate memory for each available GPU:

```c
const int num_gpus;
cudaGetDeviceCount(&num_gpus);

const uint64_t num_entries = 1UL << 26;
const uint64_t chunk_size = sdiv(num_entries, num_gpus);

uint64_t *data_gpu[num_gpus]; // One pointer for each GPU.

for (int gpu = 0; gpu < num_gpus; gpu++) {

    cudaSetDevice(gpu);

    const uint64_t lower = chunk_size*gpu;
    const uint64_t upper = min(lower+chunk_size, num_entries);
    const uint64_t width = upper-lower;

    cudaMalloc(&data_gpu[gpu], sizeof(uint64_t)*width); // Allocate chunk of data for current GPU.
}
```

<h2 style="color:#65AE11;">Data Copies for Multiple GPUs</h2>

Using the same looping and chunking techniques, data can be transfered to and from multiple GPUs:

```c
// ...Assume data has been allocated on host and for each GPU

for (int gpu = 0; gpu < num_gpus; gpu++) {

    cudaSetDevice(gpu);

    const uint64_t lower = chunk_size*gpu;
    const uint64_t upper = min(lower+chunk_size, num_entries);
    const uint64_t width = upper-lower;

    // Note use of `cudaMemcpy` and not `cudaMemcpyAsync` since we are not
    // presently using non-default streams.
    cudaMemcpy(data_gpu[gpu], data_cpu+lower, 
           sizeof(uint64_t)*width, cudaMemcpyHostToDevice); // ...or cudaMemcpyDeviceToHost
}
```

<h2 style="color:#65AE11;">Kernel Launches for Multiple GPUs</h2>

Using the same looping and chunking techniques, kernels can be launched to work on chunks of data on multiple GPUs:

```c
// ...Assume data has been allocated on host and for each GPU

for (int gpu = 0; gpu < num_gpus; gpu++) {

    cudaSetDevice(gpu);

    const uint64_t lower = chunk_size*gpu;
    const uint64_t upper = min(lower+chunk_size, num_entries);
    const uint64_t width = upper-lower;

    kernel<<<grid, block>>>(data_gpu[gpu], width); // Pass chunk of data for current GPU to work on.
}
```

<h2 style="color:#65AE11;">Check for Understanding</h2>

Please answer the following to confirm you've learned the main objectives of this section. You can display the answers for each question by clicking on the "..." cells below the questions.

---

**What CUDA Runtime call tells us how many GPUs are available?**

1. `cudaGetDevice`
2. `cudaSetDevice`
3. `cudaGetDeviceCount`
4. `cudaGetDeviceProperties`

Click to Show Solution

**Answer: 3**

---

**What CUDA Runtime selects a GPU to be currently active?**

1. `cudaGetDevice`
2. `cudaSetDevice`
3. `cudaGetDeviceCount`
4. `cudaGetDeviceProperties`

Click to Show Solution

**Answer: 2**

---

**What would the index be for a GPU on a single-GPU system?**

1. 1
2. 0

Click to Show Solution

**Answer: 2**

---

<h2 style="color:#65AE11;">Next</h2>

Now that you are familiar with the syntax and techniques for utilizing multiple GPUs, you will, in the next section, apply your understanding to refactor the baseline cipher to use multiple GPUs.

Please continue to the next section: [*Exercise: MGPU*](../11_Exercise_MGPU/Exercise_MGPU.ipynb).

<h2 style="color:#65AE11;">Optional Further Study</h2>

The following are for students with time and interest to do additional study on topics related to this workshop.

* In the above, we utlize a [depth-first](https://www.geeksforgeeks.org/difference-between-bfs-and-dfs/#:~:text=BFS(Breadth%20First%20Search)%20uses,edges%20from%20a%20source%20vertex.) approach to pass chunks of work to each GPU. In some scenarios, especially when the amount of data is extreme, it may make much more sense to utilize a breadth-first approach. This change in approach is not somehing that requires additional CUDA knowledge, but, this [stack overflow answer](https://stackoverflow.com/questions/11673154/concurrency-in-cuda-multi-gpu-executions) provides several examples of CUDA code using both depth-first and breadth-first approaches.
* Both peer to peer memory transfers between multiple GPUs, and, the use of multiple GPUs on multiple nodes are outside the scope of this workshop. [This Supercomputing Conference Presentation](https://www.nvidia.com/docs/IO/116711/sc11-multi-gpu.pdf) will give you a good starting point for exploring these topics (and more).